# Reader

We've already explored the *reader* model in the first few sections of this chapter, where we loaded a pretrained model using the `transformers` library.

Now, we'll do the same thing but via Haystack - which uses the same `transformers` library in the background (so we can use the same pretrained model names).

To initialize our reader via Haystack all we need is:

In [1]:
from haystack.reader.farm import FARMReader

reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2', use_gpu=True)

03/27/2021 14:23:42 - INFO - faiss -   Loading faiss with AVX2 support.
03/27/2021 14:23:42 - INFO - faiss -   Loading faiss.
03/27/2021 14:23:42 - INFO - farm.utils -   Using device: CUDA 
03/27/2021 14:23:42 - INFO - farm.utils -   Number of GPUs: 1
03/27/2021 14:23:42 - INFO - farm.utils -   Distributed Training: False
03/27/2021 14:23:42 - INFO - farm.utils -   Automatic Mixed Precision: None
03/27/2021 14:23:53 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
03/27/2021 14:23:53 - INFO - farm.utils -   Using device: CUDA 
03/27/2021 14:23:53 - INFO - farm.utils -   Number of GPUs: 1
03/27/2021 14:23:53 - INFO - farm.utils -   Distributed Training: False
03/27/2021 14:23:53 - INFO - farm.utils -   Automatic Mixed Precision: None
03/27/2021 14:23:54 - INFO - farm.infer -   Got ya 19 parallel workers to do inference ...
03/27/2021 14:23:54 - INFO - farm.infer -    0    0    0    0    0    0    0    0    0    0    0    

Now we have our reader model initialized, let's load up our FAISS index and DPR just like before.

In [2]:
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.retriever.dense import DensePassageRetriever

path = '../../models/faiss'

# load FAISS from file (the squad validation set index)
document_store = FAISSDocumentStore.load(f'{path}/squad_dev.faiss', f'sqlite:///{path}/squad_dev.db')

# initialize DPR model
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

Haystack comes with a very convenient `ExtractiveQAPipeline` class which allows us to pass our `reader` and `retriever` to build an easy-to-use *extractive* Q&A pipeline:

In [5]:
from haystack.pipeline import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

We can ask questions using the `run` method, alongside the `query` parameter:

In [8]:
pipeline.run(query='What does theoretical computer science cover?')

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 99.10 Batches/s]


{'query': 'What does theoretical computer science cover?',
 'no_ans_gap': 6.868406772613525,
 'answers': [{'answer': 'analysis of algorithms and computability theory',
   'score': 13.122962951660156,
   'probability': 0.8375853272849054,
   'context': ' related fields in theoretical computer science are analysis of algorithms and computability theory. A key distinction between analysis of algorithms ',
   'offset_start': 52,
   'offset_end': 99,
   'offset_start_in_doc': 59,
   'offset_end_in_doc': 106,
   'document_id': '285be3e8-24c2-43f3-bb09-04d0847a9c13',
   'meta': {'vector_id': '177'}},
  {'answer': 'Science of the Physical Universe',
   'score': 6.1175031661987305,
   'probability': 0.6823706602798535,
   'context': 'l Reasoning, Ethical Reasoning, Science of Living Systems, Science of the Physical Universe, Societies of the World, and United States in the World. H',
   'offset_start': 59,
   'offset_end': 91,
   'offset_start_in_doc': 556,
   'offset_end_in_doc': 588,
   'docu

Here we're returning a lot of different answers, which is not really necessary. We can limit the number of answers we return using two parameters, `top_k_retriever` and `top_k_reader`. Both of these limit the number of items being returned from the `retriever` and `reader` models respectively.

In [9]:
pipeline.run(query='What does theoretical computer science cover?',
             top_k_retriever=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 79.59 Batches/s]


{'query': 'What does theoretical computer science cover?',
 'no_ans_gap': 6.868406772613525,
 'answers': [{'answer': 'analysis of algorithms and computability theory',
   'score': 13.122962951660156,
   'probability': 0.8375853272849054,
   'context': ' related fields in theoretical computer science are analysis of algorithms and computability theory. A key distinction between analysis of algorithms ',
   'offset_start': 52,
   'offset_end': 99,
   'offset_start_in_doc': 59,
   'offset_end_in_doc': 106,
   'document_id': '285be3e8-24c2-43f3-bb09-04d0847a9c13',
   'meta': {'vector_id': '177'}},
  {'answer': 'Science of the Physical Universe',
   'score': 6.1175031661987305,
   'probability': 0.6823706602798535,
   'context': 'l Reasoning, Ethical Reasoning, Science of Living Systems, Science of the Physical Universe, Societies of the World, and United States in the World. H',
   'offset_start': 59,
   'offset_end': 91,
   'offset_start_in_doc': 556,
   'offset_end_in_doc': 588,
   'docu

And the `reader`:

In [10]:
pipeline.run(query='What does theoretical computer science cover?',
             top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 79.24 Batches/s]


{'query': 'What does theoretical computer science cover?',
 'no_ans_gap': 6.868406772613525,
 'answers': [{'answer': 'analysis of algorithms and computability theory',
   'score': 13.122962951660156,
   'probability': 0.8375853272849054,
   'context': ' related fields in theoretical computer science are analysis of algorithms and computability theory. A key distinction between analysis of algorithms ',
   'offset_start': 52,
   'offset_end': 99,
   'offset_start_in_doc': 59,
   'offset_end_in_doc': 106,
   'document_id': '285be3e8-24c2-43f3-bb09-04d0847a9c13',
   'meta': {'vector_id': '177'}},
  {'answer': 'Science of the Physical Universe',
   'score': 6.1175031661987305,
   'probability': 0.6823706602798535,
   'context': 'l Reasoning, Ethical Reasoning, Science of Living Systems, Science of the Physical Universe, Societies of the World, and United States in the World. H',
   'offset_start': 59,
   'offset_end': 91,
   'offset_start_in_doc': 556,
   'offset_end_in_doc': 588,
   'docu